# Nature CNN Recreation

In [1]:
%cd ..

c:\Users\Gebruiker\Documents\YEAR 3\Neural Networks\traffic-sign-classification


## Data

In [2]:
from nn.utils import one_hot_encode, export_encoded_labels
from dataio.gtsrb_dataset import GTSRBDataset
import numpy as np

chosen_classes = {1, 2, 10, 13, 38}

dataset = GTSRBDataset()
labels = dataset.labels_data

filtered_indices = [i for i, (_, label) in enumerate(labels) if label in chosen_classes]

class_indices = {cls: [] for cls in chosen_classes}
for idx in filtered_indices:
    _, label = labels[idx]
    class_indices[label].append(idx)

clipped_indices = []
for cls in chosen_classes:
    indices = class_indices[cls]
    if len(indices) > 2500:
        indices = np.random.choice(indices, size=2500, replace=False)
    clipped_indices.extend(indices)

clipped_indices = np.array(clipped_indices)
np.random.shuffle(clipped_indices)

label_to_index = {label: idx for idx, label in enumerate(chosen_classes)}
mapped_labels = np.array([label_to_index[labels[i][1]] for i in clipped_indices])

num_classes = len(chosen_classes)
one_hot_labels = one_hot_encode(mapped_labels, num_classes)

filtered_csv_path = export_encoded_labels(
    dataset=GTSRBDataset(),
    chosen_classes=chosen_classes,
    clipped_indices=clipped_indices,
    output_name="filtered_labels_encoded.csv"
)

filtered_dataset = GTSRBDataset(labels="filtered_labels_encoded.csv")
num_samples = len(filtered_dataset)

train_end = int(0.85 * num_samples)
train_split = list(range(train_end))
test_split = list(range(train_end, num_samples))

In [3]:
from dataio.transforms import ToCompose, ToGrayscale, ToResize, ToTensor, ToNormalize
from dataio.dataloader import DataLoader

# Set up the data transformations to match the original nature CNN.
train_transforms = ToCompose([
    ToGrayscale(),
    ToResize(size=28),
    ToTensor(),
    ToNormalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])
val_transforms = ToCompose([
    ToGrayscale(),
    ToResize(size=28),
    ToTensor(),
    ToNormalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

## Architecture

In [4]:
from nn.layers.conv2d import Conv2D
from nn.layers.flatten import Flatten
from nn.layers.linear import Linear
from nn.layers.maxpool2d import MaxPool2D
from nn.layers.sequential import Sequential
from nn.layers.activation import Activation
from nn.activations import relu
from nn.__init__ import he_init


In [5]:
from nn.loss import mse

# Define the loss function.
# The original CNN used MSE.
loss_fn = mse

## Hyperparameter search

In [6]:
from crossval import cross_validate
from nn.optim import Momentum

# Perform cross-validation for hyperparameter tuning.
num_epochs = 10
hyperparameters = [
    {"lr": 0.001, "wd": 1e-4},
    {"lr": 0.005, "wd": 5e-4},
    {"lr": 0.01, "wd": 1e-3},
]
all_results = []

def make_optimizer(model_params, lr: float, wd: float):
    return Momentum(model_params, lr=lr, weight_decay=wd)


def make_model():
    return Sequential([
    Conv2D(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0, weight_init=he_init),
    Activation(relu),
    MaxPool2D(pool_size=2, stride=2),
    Conv2D(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0, weight_init=he_init),
    Activation(relu),
    MaxPool2D(pool_size=2, stride=2),
    Flatten(),
    Linear(in_features=256, out_features=120, weight_init=he_init),
    Activation(relu),
    Linear(in_features=120, out_features=84, weight_init=he_init),
    Activation(relu),
    Linear(in_features=84, out_features=5, weight_init=he_init)
])


for params in hyperparameters:
    print(f"Testing hyperparameters: lr={params['lr']}, wd={params['wd']}")
    model = make_model()

    results = cross_validate(
        model_fn=make_model,
        dataset=filtered_dataset,
        loss_fn=loss_fn,
        optimizer_fn=lambda p: make_optimizer(p, params['lr'], params['wd']),
        train_transforms=train_transforms,
        val_transforms=val_transforms,
        num_epochs=num_epochs,
        batch_size=64
    )

    all_results.append({
        "lr": params['lr'],
        "wd": params['wd'],
        "fold_results": results
    })

Testing hyperparameters: lr=0.001, wd=0.0001


2025-11-03 13:12:35,419 - root - INFO - Epoch 1/10, Train Loss: 1.2345, Train Accuracy: 0.2011
2025-11-03 13:12:35,419 - root - INFO - Epoch 1/10, Train Loss: 1.2345, Train Accuracy: 0.2011
2025-11-03 13:12:44,002 - root - INFO - Epoch 1/10, Val Loss: 1.2447, Val Accuracy: 0.2078
2025-11-03 13:12:44,002 - root - INFO - Epoch 1/10, Val Loss: 1.2447, Val Accuracy: 0.2078
2025-11-03 13:14:03,132 - root - INFO - Epoch 2/10, Train Loss: 1.2424, Train Accuracy: 0.1989
2025-11-03 13:14:03,132 - root - INFO - Epoch 2/10, Train Loss: 1.2424, Train Accuracy: 0.1989
2025-11-03 13:14:10,761 - root - INFO - Epoch 2/10, Val Loss: 1.2447, Val Accuracy: 0.2078
2025-11-03 13:14:10,761 - root - INFO - Epoch 2/10, Val Loss: 1.2447, Val Accuracy: 0.2078
2025-11-03 13:15:27,571 - root - INFO - Epoch 3/10, Train Loss: 1.2431, Train Accuracy: 0.1975
2025-11-03 13:15:27,571 - root - INFO - Epoch 3/10, Train Loss: 1.2431, Train Accuracy: 0.1975
2025-11-03 13:15:35,212 - root - INFO - Epoch 3/10, Val Loss: 1.24

Testing hyperparameters: lr=0.005, wd=0.0005


2025-11-03 14:24:06,059 - root - INFO - Epoch 1/10, Train Loss: 0.9743, Train Accuracy: 0.1989
2025-11-03 14:24:06,059 - root - INFO - Epoch 1/10, Train Loss: 0.9743, Train Accuracy: 0.1989
2025-11-03 14:24:06,059 - root - INFO - Epoch 1/10, Train Loss: 0.9743, Train Accuracy: 0.1989
2025-11-03 14:24:06,059 - root - INFO - Epoch 1/10, Train Loss: 0.9743, Train Accuracy: 0.1989
2025-11-03 14:24:06,059 - root - INFO - Epoch 1/10, Train Loss: 0.9743, Train Accuracy: 0.1989
2025-11-03 14:24:06,059 - root - INFO - Epoch 1/10, Train Loss: 0.9743, Train Accuracy: 0.1989
2025-11-03 14:24:06,059 - root - INFO - Epoch 1/10, Train Loss: 0.9743, Train Accuracy: 0.1989
2025-11-03 14:24:06,059 - root - INFO - Epoch 1/10, Train Loss: 0.9743, Train Accuracy: 0.1989
2025-11-03 14:24:06,059 - root - INFO - Epoch 1/10, Train Loss: 0.9743, Train Accuracy: 0.1989
2025-11-03 14:24:06,059 - root - INFO - Epoch 1/10, Train Loss: 0.9743, Train Accuracy: 0.1989
2025-11-03 14:24:06,059 - root - INFO - Epoch 1/10

Testing hyperparameters: lr=0.01, wd=0.001


2025-11-03 15:35:16,450 - root - INFO - Epoch 1/10, Train Loss: 2.5675, Train Accuracy: 0.1986
2025-11-03 15:35:16,450 - root - INFO - Epoch 1/10, Train Loss: 2.5675, Train Accuracy: 0.1986
2025-11-03 15:35:16,450 - root - INFO - Epoch 1/10, Train Loss: 2.5675, Train Accuracy: 0.1986
2025-11-03 15:35:16,450 - root - INFO - Epoch 1/10, Train Loss: 2.5675, Train Accuracy: 0.1986
2025-11-03 15:35:16,450 - root - INFO - Epoch 1/10, Train Loss: 2.5675, Train Accuracy: 0.1986
2025-11-03 15:35:16,450 - root - INFO - Epoch 1/10, Train Loss: 2.5675, Train Accuracy: 0.1986
2025-11-03 15:35:16,450 - root - INFO - Epoch 1/10, Train Loss: 2.5675, Train Accuracy: 0.1986
2025-11-03 15:35:16,450 - root - INFO - Epoch 1/10, Train Loss: 2.5675, Train Accuracy: 0.1986
2025-11-03 15:35:16,450 - root - INFO - Epoch 1/10, Train Loss: 2.5675, Train Accuracy: 0.1986
2025-11-03 15:35:16,450 - root - INFO - Epoch 1/10, Train Loss: 2.5675, Train Accuracy: 0.1986
2025-11-03 15:35:16,450 - root - INFO - Epoch 1/10

## Result Evaluation and Greedy Search

In [8]:
for res in all_results:
    val_accs = [fold["val_acc"] for fold in res["fold_results"]]
    val_losses = [fold["val_loss"] for fold in res["fold_results"]]
    
    res["mean_val_acc"] = np.mean(val_accs)
    res["mean_val_loss"] = np.mean(val_losses)
    
    print(f"lr={res['lr']}, wd={res['wd']}: "
          f"Mean Val Acc={res['mean_val_acc']:.4f}, "
          f"Mean Val Loss={res['mean_val_loss']:.4f}")
    
best = max(all_results, key=lambda r: r["mean_val_acc"])
print("Best hyperparameters:")
print(f"lr={best['lr']}, wd={best['wd']}, Mean Val Acc={best['mean_val_acc']:.4f}")

lr=0.001, wd=0.0001: Mean Val Acc=0.2041, Mean Val Loss=1.9268
lr=0.005, wd=0.0005: Mean Val Acc=0.2008, Mean Val Loss=1.4309
lr=0.01, wd=0.001: Mean Val Acc=0.2053, Mean Val Loss=1.1184
Best hyperparameters:
lr=0.01, wd=0.001, Mean Val Acc=0.2053
